In [4]:
# 1. Derive an Edge Detection Kernel
import torch

# Directional vector
v1 = 2
v2 = 3

# Compute the angle between the directional vector and the x-axis
theta = torch.atan2(torch.tensor(v2), torch.tensor(v1))

# Define the Sobel filter kernels for x-axis and y-axis
sobel_x = torch.tensor([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=torch.float)

sobel_y = torch.tensor([
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]
], dtype=torch.float)

# Rotate the Sobel filter kernels by the computed angle
def rotate_kernel(kernel, angle):
    rotated_kernel = torch.rot90(kernel, -int(angle.item() / 90))
    return rotated_kernel

# Compute the rotated Sobel filter kernels
rotated_kernel_x = rotate_kernel(sobel_x, theta)
rotated_kernel_y = rotate_kernel(sobel_y, theta)

# Print the rotated Sobel filter kernels
print("Rotated Sobel filter kernel for x-axis:")
print(rotated_kernel_x)
print("\nRotated Sobel filter kernel for y-axis:")
print(rotated_kernel_y)


Rotated Sobel filter kernel for x-axis:
tensor([[-1.,  0.,  1.],
        [-2.,  0.,  2.],
        [-1.,  0.,  1.]])

Rotated Sobel filter kernel for y-axis:
tensor([[-1., -2., -1.],
        [ 0.,  0.,  0.],
        [ 1.,  2.,  1.]])


In [5]:
# 2. Derive a finite difference operator for the second derivative. What is the minimum size
# of the convolutional kernel associated with it? Which structures in images respond
# most strongly to it?

def second_derivative(input_signal, spacing):
    """
    Compute the second derivative of a discrete signal using finite difference approximation.

    Args:
    - input_signal (Tensor): 1D tensor representing the input signal.
    - spacing (float): Spacing between adjacent points.

    Returns:
    - second_derivative (Tensor): 1D tensor representing the second derivative of the input signal.
    """
    # Shift the input signal to compute the differences
    shifted_right = torch.cat([input_signal[1:], input_signal[-1].unsqueeze(0)], dim=0)
    shifted_left = torch.cat([input_signal[0].unsqueeze(0), input_signal[:-1]], dim=0)

    # Compute the second derivative using finite differences
    second_derivative = (shifted_right - 2 * input_signal + shifted_left) / (spacing ** 2)
    
    return second_derivative

# Example usage
input_signal = torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32)
spacing = 1.0  # Assuming spacing between adjacent points is 1
second_derivative_result = second_derivative(input_signal, spacing)
print("Second derivative:", second_derivative_result)


# The minimum size of the convolutional kernel associated with it would be 
# 3×3. This is because we need at least three points to compute the second derivative using a centered finite difference scheme.

# In image processing, structures that respond most strongly to the second derivative operator are edges and corners. 
# This is because the second derivative measures the rate of change of the gradient, which is high at edges and corners.

Second derivative: tensor([ 1.,  0.,  0.,  0., -1.])


In [6]:
# 3. How would you design a blur kernel? Why might you want to use such a kernel?

import torch.nn.functional as F
import numpy as np

def gaussian_kernel(kernel_size=3, sigma=1.0):
    """
    Generates a 2D Gaussian kernel.

    Args:
    - kernel_size (int): Size of the kernel (should be odd).
    - sigma (float): Standard deviation of the Gaussian distribution.

    Returns:
    - kernel (Tensor): 2D tensor representing the Gaussian kernel.
    """
    # Create a 1D Gaussian kernel along the x-axis
    kernel_1d = torch.tensor([np.exp(-(x - (kernel_size - 1) / 2) ** 2 / (2 * sigma ** 2)) for x in range(kernel_size)], dtype=torch.float32)
    
    # Normalize the kernel
    kernel_1d /= torch.sum(kernel_1d)
    
    # Create a 2D Gaussian kernel by taking the outer product of the 1D kernel
    kernel = torch.ger(kernel_1d, kernel_1d)
    
    return kernel

# Example usage
kernel_size = 5
sigma = 1.0
blur_kernel = gaussian_kernel(kernel_size, sigma)
print("Blur kernel:")
print(blur_kernel)

# May want to use a blurring kernel for image smoothing/blurring, image downsampling, and image deblurring.

Blur kernel:
tensor([[0.0030, 0.0133, 0.0219, 0.0133, 0.0030],
        [0.0133, 0.0596, 0.0983, 0.0596, 0.0133],
        [0.0219, 0.0983, 0.1621, 0.0983, 0.0219],
        [0.0133, 0.0596, 0.0983, 0.0596, 0.0133],
        [0.0030, 0.0133, 0.0219, 0.0133, 0.0030]])


In [7]:
# 4. What is the minimum size of a kernel to obtain a derivative of order d?

# For a derivative of order d, the minimum size of the kernel would be d + 1. This is because we need at least d+1 points 
# to compute a derivative of order d using finite differences.

# Example Code

def derivative_kernel(order):
    """
    Design a kernel for computing a derivative of a given order.

    Args:
    - order (int): Order of the derivative.

    Returns:
    - kernel (Tensor): 1D tensor representing the derivative kernel.
    """
    # Constructing the kernel coefficients using the binomial coefficients
    binomial_coeffs = torch.tensor([1, -1], dtype=torch.float32)
    kernel = binomial_coeffs
    for i in range(1, order):
        kernel = torch.conv1d(kernel.unsqueeze(0), binomial_coeffs.unsqueeze(0), padding=i)
    return kernel

# Example usage
order = 1  # Example: first-order derivative
kernel = derivative_kernel(order)
print("Derivative kernel:", kernel)


Derivative kernel: tensor([ 1., -1.])
